In [1]:
#change Kernel to "...tensorflow_27
#we are importing packages

import scipy.io
import scipy.misc
import tensorflow as tf
import numpy as np
import time
import os
from IPython.display import Image

from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave #to be able to save as an image
from scipy.optimize import fmin_l_bfgs_b #to optimize our cost function

from keras.applications import vgg19 #to load vgg19 network
from keras import backend as K

Using TensorFlow backend.


In [16]:
import keras
print(tf.__version__)
print(keras.__version__)

from keras.utils.vis_utils import plot_model #to be able to visualize the network
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

1.4.0
2.1.2


In [18]:
# let's load images

content_image_path = './SourceImg/'
style_reference_image_path = './StyleImg/'


contentList = os.listdir(content_image_path)
if ".ipynb_checkpoints" in contentList:
    contentList.remove(".ipynb_checkpoints")
contentList.sort(key=str.lower)


target_image_ref_name = contentList[0]

styleList = os.listdir(style_reference_image_path)
if ".ipynb_checkpoints" in styleList:
    styleList.remove(".ipynb_checkpoints")
styleList.sort(key=str.lower)

styleStack = ""

print(styleList)

# get dimensions (width, height) of the generated picture
width, height = load_img(content_image_path + target_image_ref_name).size

img_nrows = 400 #recale the image to 400 pixel rows 
img_ncols = int(width * img_nrows / height)


['01_vangoghmuseum-s0031V1962-800.jpg', 'abstract-purple-onion-flowers-color-color.jpg', 'Stained_Glass_Disney.jpg']


In [4]:
iterations = 20 #we will run 20 iterations during the optiomization. each iteration you get better value. this one is for about 15 min. running. 

#style_iterations = [10,6,8]
#if len(style_iterations) != len(styleList):
#    print("--------------------------------------------------")
#    print("Numbers of iteration do not match numbers of style")
#    print("--------------------------------------------------")

# these are the weights of the different loss components
total_variation_weight = 1.0 
style_weight = 1.0 #the best number authors got from their trials 
content_weight = 0.025 #the best number authors got from their trials 

In [5]:
# pre-process the image: rescaling, running it through VGG19

def preprocess_image(image_path):

    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img


# utility function to convert a tensor into a valid image

def deprocess_image(x):

    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))

    else:

        x = x.reshape((img_nrows, img_ncols, 3))

    # Remove zero-center by mean pixel

    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # 'BGR'->'RGB'

    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

#calculating the mean pixel values of the VGG network. We will use these values for normalization.
#mean = data['meta'][0][0][2][0][0][2]
#mean_pixel = np.mean(mean, axis=(0, 1))
#print(mean_pixel)

In [6]:
# compute the neural style loss
# first we need to define 4 utility functions
# the gram matrix of an image tensor (feature-wise outer product)

def gram_matrix(x):

    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))

    gram = K.dot(features, K.transpose(features))
    return gram

In [7]:
# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image

def style_loss(style, combination):

    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3

    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


In [8]:
# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image

def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [9]:
# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent (no big changes)

def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':

        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [10]:
def eval_loss_and_grads(x):

    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))

    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values



In [11]:
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [12]:
evaluator = Evaluator()

In [13]:
for style in styleList:
    
    #iterations = style_iterations.pop(0)
    styleAddr = style_reference_image_path + style
    image_style = scipy.misc.imread(styleAddr)
    
    #image_content= scipy.misc.imresize(image_content,(224,224))
    #plt.figure(figsize=(11,11))
    #plt.imshow(image_style)
    
    style_reference_image = K.variable(preprocess_image(styleAddr))
    styleName = style[:style.find(".")]
    styleStack += styleName + '_'

    for content in contentList:
        
        fileAddr = content_image_path + content
        contentName = content[:content.find(".")]


        image_content = scipy.misc.imread(fileAddr)
        

        #image_content= scipy.misc.imresize(image_content,(224,224))
        #plt.figure(figsize=(11,11))
        #plt.imshow(image_content)

        # get tensor representations of our images

        base_image = K.variable(preprocess_image(fileAddr))

        # this will contain our generated image
        if K.image_data_format() == 'channels_first':
            combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
        else:
            combination_image = K.placeholder((1, img_nrows, img_ncols, 3))


        # combine the 3 images (style, content, result image that starts from the white noise) into a single Keras tensor

        input_tensor = K.concatenate([base_image,
                                      style_reference_image,
                                      combination_image], axis=0)

        # build the VGG19 network with our 3 images as input
        # the model will be loaded with pre-trained ImageNet weights

        model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet', include_top=False)
        print('Model loaded.')

        #plot_model(model, show_shapes=True, to_file='./images/model.png')
        #Image("./images/model.png")
        #for l, layer in enumerate(model.layers):
        #print(layer.name)
        # get the symbolic outputs of each "key" layer (we gave them unique names).
        outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
        # combine these loss functions into a single scalar

        loss = K.variable(0.)
        layer_features = outputs_dict['block5_conv2'] #content features
        base_image_features = layer_features[0, :, :, :]
        combination_features = layer_features[2, :, :, :]
        loss += content_weight * content_loss(base_image_features,
                                              combination_features)

        # style_features
        feature_layers = ['block1_conv1', 'block2_conv1',
                          'block3_conv1', 'block4_conv1',
                          'block5_conv1']

        for layer_name in feature_layers:
            layer_features = outputs_dict[layer_name]
            style_reference_features = layer_features[1, :, :, :]
            combination_features = layer_features[2, :, :, :]
            sl = style_loss(style_reference_features, combination_features)
            loss += (style_weight / len(feature_layers)) * sl

        loss += total_variation_weight * total_variation_loss(combination_image)

        # get the gradients of the generated image wrt the loss

        grads = K.gradients(loss, combination_image)
        outputs = [loss]

        if isinstance(grads, (list, tuple)):
            outputs += grads
        else:
            outputs.append(grads)

        f_outputs = K.function([combination_image], outputs)

        # run scipy-based optimization (L-BFGS) over the pixels of the generated image
        # so as to minimize the neural style loss
        x = preprocess_image(fileAddr)

        
        result_prefix ='./Intermediate_Result/' + styleStack + contentName
        # minimise the loss function

        for i in range(iterations):

            print('Start of iteration', i)
            start_time = time.time()

            x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                             fprime=evaluator.grads, maxfun=20)

            print('Current loss value:', min_val)
            # save current generated image

            img = deprocess_image(x.copy())
            fname = result_prefix + '_at_iteration_%d.png' % i
            imsave(fname, img)
            imsave(fileAddr, img)
            end_time = time.time()

            print('Image saved as', fname)
            print('Iteration %d completed in %ds' % (i, end_time - start_time))

            #image_content= scipy.misc.imresize(image_content,(224,224))
            #plt.figure(figsize=(11,11))
            #plt.imshow(img)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Model loaded.
Start of iteration 0
Current loss value: 1.98785e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_0.png
Iteration 0 completed in 54s
Start of iteration 1


/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:113: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:114: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 1.32198e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_1.png
Iteration 1 completed in 16s
Start of iteration 2
Current loss value: 1.10643e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_2.png
Iteration 2 completed in 16s
Start of iteration 3
Current loss value: 1.00779e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_3.png
Iteration 3 completed in 16s
Start of iteration 4
Current loss value: 9.44957e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_4.png
Iteration 4 completed in 16s
Start of iteration 5
Current loss value: 8.92216e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_at_iteration_5.png
Iteration 5 completed in 17s
Start of iteration 6
Current loss value: 8.41214e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 1_a

Current loss value: 7.91991e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 13_at_iteration_7.png
Iteration 7 completed in 17s
Start of iteration 8
Current loss value: 7.53249e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 13_at_iteration_8.png
Iteration 8 completed in 17s
Start of iteration 9
Current loss value: 7.17814e+08
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 13_at_iteration_9.png
Iteration 9 completed in 17s
Model loaded.
Start of iteration 0
Current loss value: 1.95102e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 14_at_iteration_0.png
Iteration 0 completed in 17s
Start of iteration 1
Current loss value: 1.28097e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Layer 14_at_iteration_1.png
Iteration 1 completed in 17s
Start of iteration 2
Current loss value: 1.08501e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031

...

Model loaded.
Start of iteration 0
Current loss value: 4.16877e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_Layer 9_at_iteration_0.png
Iteration 0 completed in 19s
Start of iteration 1
Current loss value: 2.65951e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_Layer 9_at_iteration_1.png
Iteration 1 completed in 17s
Start of iteration 2
Current loss value: 2.25864e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_Layer 9_at_iteration_2.png
Iteration 2 completed in 17s
Start of iteration 3
Current loss value: 2.1043e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_Layer 9_at_iteration_3.png
Iteration 3 completed in 17s
Start of iteration 4
Current loss value: 1.99128e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_Layer 9_at_iteration_4.png
Iteration 4 completed

Current loss value: 4.76712e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 12_at_iteration_5.png
Iteration 5 completed in 17s
Start of iteration 6
Current loss value: 4.56917e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 12_at_iteration_6.png
Iteration 6 completed in 17s
Start of iteration 7
Current loss value: 4.41524e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 12_at_iteration_7.png
Iteration 7 completed in 17s
Model loaded.
Start of iteration 0
Current loss value: 1.11982e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 13_at_iteration_0.png
Iteration 0 completed in 19s
Start of iteration 1
Current loss value

Current loss value: 4.08059e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 16_at_iteration_7.png
Iteration 7 completed in 17s
Model loaded.
Start of iteration 0
Current loss value: 1.12307e+10
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 17_at_iteration_0.png
Iteration 0 completed in 20s
Start of iteration 1
Current loss value: 6.97209e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 17_at_iteration_1.png
Iteration 1 completed in 17s
Start of iteration 2
Current loss value: 5.80204e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 17_at_iteration_2.png
Iteration 2 completed in 17s
Start of iteration 3
Current loss value

Current loss value: 7.03626e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 5_at_iteration_1.png
Iteration 1 completed in 17s
Start of iteration 2
Current loss value: 5.83113e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 5_at_iteration_2.png
Iteration 2 completed in 17s
Start of iteration 3
Current loss value: 5.24325e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 5_at_iteration_3.png
Iteration 3 completed in 17s
Start of iteration 4
Current loss value: 4.88717e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 5_at_iteration_4.png
Iteration 4 completed in 17s
Start of iteration 5
Current loss value: 4.64331e+09
Imag

Current loss value: 5.25571e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 9_at_iteration_3.png
Iteration 3 completed in 17s
Start of iteration 4
Current loss value: 4.90642e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 9_at_iteration_4.png
Iteration 4 completed in 17s
Start of iteration 5
Current loss value: 4.66675e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 9_at_iteration_5.png
Iteration 5 completed in 17s
Start of iteration 6
Current loss value: 4.47993e+09
Image saved as ./Intermediate_Result/01_vangoghmuseum-s0031V1962-800_Stained_Glass_Disney_abstract-purple-onion-flowers-color-color_Layer 9_at_iteration_6.png
Iteration 6 completed in 17s
Start of iteration 7
Current loss value: 4.33882e+09
Imag